In [37]:
from dotenv import load_dotenv
from huggingface_hub import InferenceClient
from qdrant_client import QdrantClient
from qdrant_client.http import models
import os
import re

In [38]:
load_dotenv()

client = InferenceClient(
    provider="hf-inference",
    api_key=os.environ["HF_TOKEN"],
)

database = QdrantClient(
    url=os.environ["QDRANT_URL"],
    api_key=os.environ["QDRANT_API_KEY"],
)

In [39]:
database.recreate_collection(
    collection_name="PassportKnowledgeBase",
    vectors_config={
        "chunk_vector": models.VectorParams(size=768, distance=models.Distance.COSINE),
        "summary_vector": models.VectorParams(size=768, distance=models.Distance.COSINE),
        "title_vector": models.VectorParams(size=768, distance=models.Distance.COSINE)
    },
    on_disk_payload=True
)

database.create_payload_index(
    collection_name="PassportKnowledgeBase",
    field_name="summary",
    field_schema=models.PayloadSchemaType.KEYWORD
)

database.create_payload_index(
    collection_name="PassportKnowledgeBase",
    field_name="title",
    field_schema=models.PayloadSchemaType.KEYWORD
)

C:\Users\OSHITH\AppData\Local\Temp\ipykernel_5748\4263317928.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  database.recreate_collection(


UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [40]:
def clean_text(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'#{1,6}\s*', '', text)
    text = re.sub(r'(\*\*|__)(.*?)\1', r'\2', text)
    text = re.sub(r'(\*|_)(.*?)\1', r'\2', text)
    text = re.sub(r'\|', ' ', text)
    text = re.sub(r'-{3,}', '', text)
    text = re.sub(r'={3,}', '', text)
    text = re.sub(r'[\*=\-_]{3,}', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('\\n', ' ').replace('\\t', ' ')
    text = text.strip()
    return text

chunkTopics = [
    """
Business Visa (via ETA)## Eligibility Criteria
    """,
    """
Business Visa (via ETA)## Required Documents
    """,
    """
Business Visa (via ETA)## Step-by-Step Procedure
    """,
    """
Business Visa (via ETA)## Fees & Processing Time
    """,
    """
Business Visa (via ETA)## Contact Information
    """,
    """
Business Visa (via ETA)## Common Issues & Solutions
   """,
    """
    Dual Citizenship## Eligibility Criteria
""",
"""
Dual Citizenship## Required Documents
""",
"""
Dual Citizenship## Step-by-Step Procedure
""",
"""
Dual Citizenship## Fees & Processing Time
""",
"""
Dual Citizenship## Contact Information
""",
"""
Dual Citizenship## Common Issues & Solutions
""",
"""
Issue of New Ordinary Passport (All Countries)## Eligibility Criteria
""",
"""
Issue of New Ordinary Passport (All Countries)## Required Documents
""",
"""
Issue of New Ordinary Passport (All Countries)## Step-by-Step Procedure
""",
"""
Issue of New Ordinary Passport (All Countries)## Fees & Processing Time
""",
"""
Issue of New Ordinary Passport (All Countries)## Contact Information
""",
"""
Issue of New Ordinary Passport (All Countries)## Common Issues & Solutions
""",
"""
Issue of Passport (Minor Applicants - Under 16)## Eligibility Criteria
""",
"""
Issue of Passport (Minor Applicants - Under 16)## Required Documents
""",
"""
Issue of Passport (Minor Applicants - Under 16)## Step-by-Step Procedure
""",
"""
Issue of Passport (Minor Applicants - Under 16)## Fees & Processing Time
""",
"""
Issue of Passport (Minor Applicants - Under 16)## Contact Information
""",
"""
Issue of Passport (Minor Applicants - Under 16)## Common Issues & Solutions
""",
"""
My Dream Home Visa Programme (Extension Only)## Eligibility Criteria
""",
"""
My Dream Home Visa Programme (Extension Only)## Required Documents
""","""
My Dream Home Visa Programme (Extension Only)## Step-by-Step Procedure
""","""
My Dream Home Visa Programme (Extension Only)## Fees & Processing Time
""","""
My Dream Home Visa Programme (Extension Only)## Contact Information
""","""
My Dream Home Visa Programme (Extension Only)## Common Issues & Solutions
""","""
Overseas Birth Registration (for children of Sri Lankan parents)## Eligibility Criteria
""","""
Overseas Birth Registration (for children of Sri Lankan parents)## Required Documents
""","""
Overseas Birth Registration (for children of Sri Lankan parents)## Step-by-Step Procedure
""","""
Overseas Birth Registration (for children of Sri Lankan parents)## Fees & Processing Time
""","""
Overseas Birth Registration (for children of Sri Lankan parents)## Contact Information
""","""
Overseas Birth Registration (for children of Sri Lankan parents)## Common Issues & Solutions
""","""
Renunciation of Citizenship of Sri Lanka## Eligibility Criteria
""","""
Renunciation of Citizenship of Sri Lanka## Required Documents
""","""
Renunciation of Citizenship of Sri Lanka## Step-by-Step Procedure
""","""
Renunciation of Citizenship of Sri Lanka## Fees & Processing Time
""","""
Renunciation of Citizenship of Sri Lanka## Contact Information
""","""
Renunciation of Citizenship of Sri Lanka## Common Issues & Solutions
""","""
Reporting a Lost or Stolen Passport## Eligibility Criteria
""","""
Reporting a Lost or Stolen Passport## Required Documents
""","""
Reporting a Lost or Stolen Passport## Step-by-Step Procedure
""","""
Reporting a Lost or Stolen Passport## Fees & Processing Time
""","""
Reporting a Lost or Stolen Passport## Contact Information
""","""
Reporting a Lost or Stolen Passport## Common Issues & Solutions
""","""
Residence Visa (Education Category)## Eligibility Criteria
""","""
Residence Visa (Education Category)## Required Documents
""","""
Residence Visa (Education Category)## Step-by-Step Procedure
""","""
Residence Visa (Education Category)## Fees & Processing Time
""","""
Residence Visa (Education Category)## Contact Information
""","""
Residence Visa (Education Category)## Common Issues & Solutions
""","""
Residence Visa (Employment Category)## Eligibility Criteria
""","""
Residence Visa (Employment Category)## Required Documents
""","""
Residence Visa (Employment Category)## Step-by-Step Procedure
""","""
Residence Visa (Employment Category)## Fees & Processing Time
""","""
Residence Visa (Employment Category)## Contact Information
""","""
Residence Visa (Employment Category)## Common Issues & Solutions
""","""
Residence Visa (Other Categories & Fees)## Eligibility Criteria
""","""
Residence Visa (Other Categories & Fees)## Required Documents
""","""
Residence Visa (Other Categories & Fees)## Step-by-Step Procedure
""","""
Residence Visa (Other Categories & Fees)## Fees & Processing Time
""","""
Residence Visa (Other Categories & Fees)## Contact Information
""","""
Residence Visa (Other Categories & Fees)## Common Issues & Solutions
""","""
Resident Guest Scheme Visa (Extension Only)## Eligibility Criteria
""","""
Resident Guest Scheme Visa (Extension Only)## Required Documents
""","""
Resident Guest Scheme Visa (Extension Only)## Step-by-Step Procedure
""","""
Resident Guest Scheme Visa (Extension Only)## Fees & Processing Time
""","""
Resident Guest Scheme Visa (Extension Only)## Contact Information
""","""
Resident Guest Scheme Visa (Extension Only)## Common Issues & Solutions
""","""
Tourist Visa (via ETA)## Eligibility Criteria
""","""
Tourist Visa (via ETA)## Required Documents
""","""
Tourist Visa (via ETA)## Step-by-Step Procedure
""","""
Tourist Visa (via ETA)## Fees & Processing Time
""","""
Tourist Visa (via ETA)## Contact Information
""","""
Tourist Visa (via ETA)## Common Issues & Solutions
""","""
Tourist Visa Extension## Eligibility Criteria
""","""
Tourist Visa Extension## Required Documents
""","""
Tourist Visa Extension## Step-by-Step Procedure
""","""
Tourist Visa Extension## Fees & Processing Time
* **Processing Time:** Not specified.
""","""
Tourist Visa Extension## Contact Information
""","""
Tourist Visa Extension## Common Issues & Solutions
""","""
Visa Overstay Penalties## Eligibility Criteria (Applicability)
""","""
Visa Overstay Penalties## Required Documents
""","""
Visa Overstay Penalties## Step-by-Step Procedure
""","""
Visa Overstay Penalties## Fees & Processing Time
""","""
Visa Overstay Penalties## Contact Information
""","""
Visa Overstay Penalties## Common Issues & Solutions
""","""
Yellow Fever Vaccination Certificate Requirement## Eligibility Criteria (Applicability)
""","""
Yellow Fever Vaccination Certificate Requirement## Required Documents
""","""
Yellow Fever Vaccination Certificate Requirement## Step-by-Step Procedure
""","""
Yellow Fever Vaccination Certificate Requirement## Fees & Processing Time
""","""
Yellow Fever Vaccination Certificate Requirement## Contact Information
""","""
Yellow Fever Vaccination Certificate Requirement## Common Issues & Solutions
"""
]

chunks = [
    """
Business Visa (via ETA)## Eligibility Criteria
* Foreign nationals visiting Sri Lanka for short-term business purposes, including:
    * Business meetings and negotiations
    * Conferences, seminars, and workshops
    * Short-term training programs
    * Art, music, or dance events
    * Religious events or symposiums.
* Must not engage in paid/unpaid employment or self-employment.
    """,
    """
Business Visa (via ETA)## Required Documents
1.  Valid passport.
2.  Invitation or Request letter from the relevant institution in Sri Lanka.
    """,
    """
Business Visa (via ETA)## Step-by-Step Procedure
1.  Apply for the Electronic Travel Authorization (ETA) online at the official website.
2.  Alternatively, applications can be submitted via a third party, at a Sri Lanka Overseas Mission, or at the Head Office.
3.  (Note: "On Arrival" is not listed as an option for Business ETA in the provided text).
    """,
    """
Business Visa (via ETA)## Fees & Processing Time
* **Validity:** 30 days, multiple entries.
* **Fee Structure:**
| Method | SAARC Countries | All Other Countries |
| :--- | :--- | :--- |
| Online / Third Party / Mission / Head Office | USD 30 | USD 55 |
* **Children under 12 years:** Free.
    """,
    """
Business Visa (via ETA)## Contact Information
* **Website:** `https://eta.gov.lk`
* **Office:** Department of Immigration & Emigration, "Suhurupaya", Battaramulla.
    """,
    """
Business Visa (via ETA)## Common Issues & Solutions
* **Problem:** Requiring an extension.
* **Solution:** Extensions are available. For example, extension from 30 to 180 days for a US citizen is USD 100. For other countries, 30-90 days is USD 100, and 90-180 days is USD 200.
    """,
    """
    Dual Citizenship## Eligibility Criteria
* **Resumption:** A person whose Sri Lankan citizenship has ceased (under section 19, 20, or 21 of the Citizenship Act) and who wishes to resume Sri Lankan citizenship.
* **Retention:** A person who desires to obtain citizenship in another country while intending to retain Sri Lankan citizenship.
* Applicants must qualify under one of the following categories:
    * (A) Has completed the age of 55 years.
    * (B) Possesses required educational/professional qualifications (minimum one-year Diploma or higher).
    * (C) Owns assets (land or immovable properties) in Sri Lanka worth Rs. 2.5 million or more.
    * (D) Has a fixed deposit of Rs. 2.5 million or more in a Sri Lankan commercial bank for 3 years.
    * (E) Has a fixed deposit of USD 25,000 or more in a specified bank account (NRFC/RFC/SFIDA) for 3 years.
    * (F) Has invested in a Treasury Bond (TB) or Security Investment Account (SIA) of USD 25,000 for 3 years.
    * (G) Is a spouse or unmarried child (under 22) of an applicant who meets criteria A-F.
""",
"""
Dual Citizenship## Required Documents
1.  Applicant's original Birth Certificate (or certified copy).
2.  Marriage certificate (or certified copy), if applicable.
3.  An affidavit (page 6 of the application) made before a Justice of the Peace.
4.  **For Resumption (if citizenship already ceased):**
    * Certified copy of foreign Citizenship Certificate.
    * Certified copy of the foreign passport.
    * Recent Police clearance report (not older than 12 months) from the country of foreign citizenship.
    * Certified copy of the previous Sri Lankan passport (if available).
5.  **For Retention (if intending to get foreign citizenship):**
    * Certified copy of the current Sri Lankan passport.
    * Certified copy of the permanent residence visa of the foreign country.
    * Recent Police clearance report (not older than 3 months) from the country of permanent residence.
6.  Documents supporting the chosen eligibility category (e.g., professional certificates, property deeds, bank deposit confirmations).
""",
"""
Dual Citizenship## Step-by-Step Procedure
1.  Complete the application form in "BLOCK CAPITALS" (a separate application is needed for each family member).
2.  Gather all required documents. Note: Documents must be certified by a Deputy/Assistant Controller (Citizenship) or a senior officer at a Sri Lanka Overseas Mission (rank not less than Third Secretary).
3.  Submit the duly perfected application and authenticated documents to the Department Head Office or a Sri Lanka Overseas Mission.
4.  Wait for notification. The relevant fee is only charged *after* the Minister grants approval.
5.  The Department will notify the applicant via e-mail to make the payment.
6.  Pay the fee (in US Dollars) in cash, by card, or via the online payment link. The date of payment becomes the effective date of dual citizenship.
7.  The applicant will be notified by e-mail to collect the certificate.
""",
"""
Dual Citizenship## Fees & Processing Time
* **Fee Structure:**
    * **Main Applicant:** USD 2,000
    * **Spouse of Applicant:** USD 500
    * **Unmarried child below 22:** USD 500
    * **Certified copy of certificate:** Rs. 2,000
* **Processing Time:** Not specified.
""",
"""
Dual Citizenship## Contact Information
* **Relevant Office:**
    The Controller General,
    Department of Immigration and Emigration,
    Citizenship Division,
    "Suhurupaya",
    Sri Subhuthipura Road,
    Battaramulla, Sri Lanka.
* **Email:** accit@immigration.gov.lk
* **Working Hours:** Applications are accepted from 9.00 a.m. to 2.30 p.m., Monday to Friday (except Public holidays).
""",
"""
Dual Citizenship## Common Issues & Solutions
* **Problem:** Incomplete applications.
* **Solution:** All details requested in the application must be filled in. If a particular piece of information is not applicable, write "Not relevant". Incomplete applications will be rejected.
* **Problem:** Applying as a child over the age limit.
* **Solution:** Applications for unmarried children (Category G) must be submitted before the child completes the age of 21 years and 6 months.
""",
"""
Issue of New Ordinary Passport (All Countries)## Eligibility Criteria
* Must be a Sri Lankan Citizen (by descent or registration).
""",
"""
Issue of New Ordinary Passport (All Countries)## Required Documents
1.  Completed Application Form (Form K 35 A).
2.  Photo studio acknowledgement (This is a receipt from an authorized studio, *not* a printed photo).
3.  Original Birth Certificate with a photocopy.
4.  Original National Identity Card (NIC) with a photocopy.
5.  Current passport with a photocopy of the Bio data page (if you have one).
6.  Marriage certificate with a photocopy (only if required to confirm name after marriage).
7.  Educational/professional certificates and service certificate (if the profession is to be included in the passport).
""",
"""
Issue of New Ordinary Passport (All Countries)## Step-by-Step Procedure
1.  Visit an "Authorized photo studio". The studio will take your digital photograph, send it online to the Department, and give you a "Studio Acknowledgement Note".
2.  Obtain and complete the Application Form K 35 A.
3.  Go to the Department Head Office (Battaramulla) or a Regional Office (Kandy, Matara, Vavuniya, Kurunegala, Jaffna) **in person**.
4.  Submit the completed application, the Studio Acknowledgement Note, and all required original documents with photocopies.
5.  Applicants aged 16-60 must provide fingerprints at the counter.
6.  Pay the required fee for either "Normal" or "Urgent" service.
7.  Collect the passport after the specified processing time.
""",
"""
Issue of New Ordinary Passport (All Countries)## Fees & Processing Time
* **Fee Structure:**
    * **Normal Basis:** LKR 10,000.00
    * **Urgent Basis:** LKR 20,000.00
* **Processing Time:**
    * **Normal Basis:** 30 working days
    * **Urgent Basis:** Same day
""",
"""
Issue of New Ordinary Passport (All Countries)## Contact Information
* **Relevant Office:** Passport Division (Level 01 - Normal / Level 02 - Urgent), Department of Immigration & Emigration, "Suhurupaya", Battaramulla.
* **Regional Offices:** Kandy, Matara, Vavuniya, Kurunegala, Jaffna
* **Phone:** +94 112 101 520 (for photo info), Hotline 1962
* **Working Hours:** Applications accepted from 7.00 a.m. to 2.00 p.m., Monday to Friday.
""",
"""
Issue of New Ordinary Passport (All Countries)## Common Issues & Solutions
* **Problem:** Bringing printed photographs.
* **Solution:** Printed photographs are **not accepted**. Applicants *must* get a digital photo from an Authorized photo studio, which is sent online to the Department.
* **Problem:** Applying for a child.
* **Solution:** Inclusion of children in parent's passports is no longer allowed. All children (even infants) must obtain a separate passport.
""",
"""
Issue of Passport (Minor Applicants - Under 16)## Eligibility Criteria
* Sri Lankan citizen under 16 years of age.
""",
"""
Issue of Passport (Minor Applicants - Under 16)## Required Documents
1.  Original Birth certificate of the applicant with a photocopy.
2.  Photo studio acknowledgement.
3.  Parent's Passport (with photocopies of data page & child page).
4.  If parents do not have passports submit the National Identity Card.
5.  Consent Letter of the Parents. (Either the parents or the legal guardian of the applicant should accompany)
6.  Current passport with a photocopy of Bio data page (If available)
7.  **If born outside Sri Lanka:** Consular Birth Certificate and Citizenship Registration Certificate.
8.  **If adopted:** Certificate of Adoption, The court order, and a letter from the Commissioner of Probation and Child Care.
""",
"""
Issue of Passport (Minor Applicants - Under 16)## Step-by-Step Procedure
1.  Either both parents or the legal guardian **must** accompany the minor applicant to submit the application.
2.  Follow the standard passport application procedure (digital photo, K 35 A form).
3.  Submit all required documents, including the parental consent letter.
4.  Pay the fee based on the desired validity period (3 years or 10 years).
""",
"""
Issue of Passport (Minor Applicants - Under 16)## Fees & Processing Time
* **Fees (Validity 3 years):**
    * Normal Basis - LKR. 3,000.00
    * Urgent Basis - LKR. 9,000.00
* **Fees (Validity 10 years):**
    * Normal Basis - LKR. 10,000.00
    * Urgent Basis - LKR. 20,000.00
* **Processing Time:**
    * Normal Basis: 30 working days
    * Urgent Basis: Same day
""",
"""
Issue of Passport (Minor Applicants - Under 16)## Contact Information
* Same as "Issue of New Ordinary Passport".
""",
"""
Issue of Passport (Minor Applicants - Under 16)## Common Issues & Solutions
* **Problem:** Parents are divorced.
* **Solution:** Must submit the original Divorce Certificate and the court order stating the right to the custody of the child.
* **Problem:** One or both parents are abroad.
* **Solution:** The consent letter and passport copies of the parents must be certified by the relevant Sri Lankan Mission.
""",
"""
My Dream Home Visa Programme (Extension Only)## Eligibility Criteria
* This service is **only for renewal/extension** for existing visa holders.
* Applicant must maintain a fixed deposit of USD 15,000.
* Applicant must show a monthly remittance of USD 1,500 (plus USD 750 for each dependent) for upkeep.
""",
"""
My Dream Home Visa Programme (Extension Only)## Required Documents
1.  Residence Visa application form.
2.  Two passport-size colour photographs.
3.  Copy of Passport.
4.  Certified copy of the latest Bank Statement (Fixed Deposit).
5.  Proof of source of income for the monthly remittance.
6.  Police clearance certificate from Sri Lanka.
7.  Valid medical insurance policy (applicable in Sri Lanka).
8.  Marriage certificate (if renewing for spouse).
""","""
My Dream Home Visa Programme (Extension Only)## Step-by-Step Procedure
1.  Submit the application and all supporting documents to the Controller (Visa) at the Department Head Office.
2.  Applicant must submit half-year bank statements (FD and savings) to the Controller (Visa) to prove continued eligibility.
""","""
My Dream Home Visa Programme (Extension Only)## Fees & Processing Time
* **Fee:** USD 200 per year (per person, for applicant, spouse, and dependents over 16).
* **Validity:** The visa is renewed for two years.
""","""
My Dream Home Visa Programme (Extension Only)## Contact Information
* **Office:** Controller (Visa), Department of Immigration & Emigration, "Suhurupaya", Battaramulla.
""","""
My Dream Home Visa Programme (Extension Only)## Common Issues & Solutions
* **Problem:** Withdrawing the Fixed Deposit to leave the country.
* **Solution:** The applicant must obtain prior approval from the Controller of Exchange (via the Controller General of Immigration) *before* withdrawing the funds.
""","""
Overseas Birth Registration (for children of Sri Lankan parents)## Eligibility Criteria
* Children born outside Sri Lanka.
* At least one parent must be a Sri Lankan citizen at the time of the child's birth.
* The registration should be done within one year of the birth to avoid extra fees.
""","""
Overseas Birth Registration (for children of Sri Lankan parents)## Required Documents
1.  Birth Certificate of the child (issued by the country of birth). If not in English, an English translation is required.
2.  Consular Birth Certificate (issued by the Registrar General's Department of Sri Lanka).
3.  Original Birth Certificates of parents.
4.  Original Marriage Certificate of parents.
5.  Original Travel Documents and Visas held by parents (to prove residence abroad at the time of birth).
6.  Dual Citizenship certificates (if parents are dual citizens).
7.  Citizenship by registration certificate (if a parent is a Sri Lankan citizen by registration).
""","""
Overseas Birth Registration (for children of Sri Lankan parents)## Step-by-Step Procedure
1.  Obtain the application form (from the Head Office, a Sri Lankan Mission, or by download).
2.  Submit the duly completed application with all required original documents and copies.
3.  Applications can be submitted to:
    * The Citizenship Branch at the Head Office in Battaramulla.
    * The Sri Lankan Mission in the country where the child's birth occurred.
4.  Pay the required fees at the time of submission.
5.  The Department will process the application and mail the certificate to the Sri Lankan Mission or the applicant's Sri Lankan address.
""","""
Overseas Birth Registration (for children of Sri Lankan parents)## Fees & Processing Time
* **Fee Structure:**
    * **Application within one year of birth:** Rs. 5,750
    * **Delayed application (per year of delay):** Rs. 500
    * **Duplicate certificate:** Rs. 2,000
* **Processing Time:** Not specified.
""","""
Overseas Birth Registration (for children of Sri Lankan parents)## Contact Information
* **Relevant Office:** Citizenship Division (Level 04 - Wing D), Department of Immigration and Emigration, "Suhurupaya", Battaramulla.
* **Working Hours:** Applications accepted from 9.00 a.m. to 2.30 p.m., Monday to Friday.
""","""
Overseas Birth Registration (for children of Sri Lankan parents)## Common Issues & Solutions
* **Problem:** Failure to register within one year.
* **Solution:** Registration is still possible, but an extra fee of Rs. 500 for each year of delay will be imposed.
""","""
Renunciation of Citizenship of Sri Lanka## Eligibility Criteria
* Implied for Sri Lankan citizens who are voluntarily acquiring citizenship in another country.
""","""
Renunciation of Citizenship of Sri Lanka## Required Documents
1.  Applicant's Request
2.  Form "K"
3.  Letter from the relevant foreign country confirming citizenship (or a certified copy of the foreign Passport).
4.  Birth Certificate
5.  Original Sri Lankan Passport (and photocopy)
6.  Original National Identity Card (and photocopy)
7.  Original Dual Citizenship Certificate (if available)
8.  Original 5(2) Citizenship Certificate (if available)
""","""
Renunciation of Citizenship of Sri Lanka## Step-by-Step Procedure
1.  Gather all required documents.
2.  Submit all documents to the relevant Sri Lankan Embassy or directly to the Citizenship Division of the Department.
""","""
Renunciation of Citizenship of Sri Lanka## Fees & Processing Time
* Not specified.
""","""
Renunciation of Citizenship of Sri Lanka## Contact Information
* **Relevant Office:** Citizenship Division, Department of Immigration & Emigration, "Suhurupaya", Battaramulla.
""","""
Renunciation of Citizenship of Sri Lanka## Common Issues & Solutions
* Not specified.
""","""
Reporting a Lost or Stolen Passport## Eligibility Criteria
* N/A (This is a reporting procedure for any Sri Lankan passport holder).
""","""
Reporting a Lost or Stolen Passport## Required Documents
* **To make the report (if overseas):**
    1.  Police report from the local police in that country.
    2.  Completed "stolen or lost passport complaint form" (downloadable).
* **To apply for a new passport (after reporting):**
    1.  Original of the police complaint.
    2.  If lost abroad, the temporary travel document (NMRP) used to return to Sri Lanka.
""","""
Reporting a Lost or Stolen Passport## Step-by-Step Procedure
* **If you are IN SRI LANKA:**
    1.  **Immediately** report the incident by phone (011 532 9326) or fax (011-2885358) to the Department.
    2.  Make a formal complaint to your local police station as soon as possible.
    3.  The police will transmit the details to the Department, which will cancel the passport.
* **If you are OVERSEAS:**
    1.  Make a complaint to the local police in your country of residence and obtain a Police Report.
    2.  Fill out the "stolen or lost passport complaint form".
    3.  Submit the form and the police report to the nearest Sri Lanka Diplomatic Mission.
    4.  The Mission will transmit the details to the Department in Sri Lanka for cancellation.
""","""
Reporting a Lost or Stolen Passport## Fees & Processing Time
* **Fines (payable when applying for a replacement):**
    * If passport lost **within one year** of issue: Fine of Rs. 20,000 (in addition to the new passport fee).
    * If passport lost **after one year** of issue: Fine of Rs. 15,000 (in addition to the new passport fee, if validity had not lapsed).
* **Processing Time:** The cancellation is processed upon receipt of the report.
""","""
Reporting a Lost or Stolen Passport## Contact Information
* **Relevant Office:**
    Lost / Stolen Passport Desk
    International Cooperation Unit, 3rd Floor
    Department of Immigration and Emigration
    "Suhurupaya", Battaramulla, Sri Lanka.
* **Hotline:** 011 532 9326 (8:30 a.m. to 4:00 p.m., weekdays)
""","""
Reporting a Lost or Stolen Passport## Common Issues & Solutions
* **Problem:** Finding the passport *after* reporting it lost or stolen.
* **Solution:** **Never use it.** The passport is permanently invalidated and details are shared with INTERPOL. You must still apply for a new passport. If you find it, you should mail it to the Department.
""","""
Residence Visa (Education Category)## Eligibility Criteria
* University Students.
* Students in other Government-approved Educational Institutions.
* One guardian of a student under 16 years of age.
* Volunteers.
* Internship holders.
""","""
Residence Visa (Education Category)## Required Documents
* **General:** Passport(s), Health Protection Plan (HPP) certificate, copy of passport Bio Data Page.
* **University Students:**
    1.  Recommendation from the Ministry of Higher Education.
    2.  Letter from the university.
* **Other Students (and Guardian):**
    1.  Request letter from the academic institution.
    2.  Recommendation from the relevant Embassy / High Commission.
    3.  Bank Encashment receipts (to the value of USD 1,500 for a year per person).
* **Volunteers:**
    1.  Request letter from the volunteer organization.
    2.  Recommendation from the Line Ministry.
""","""
Residence Visa (Education Category)## Step-by-Step Procedure
1.  Obtain the Residence Visa application form.
2.  Obtain the mandatory Health Protection Plan (HPP).
3.  Gather all category-specific documents.
4.  Submit the application to the Department of Immigration & Emigration, Colombo.
""","""
Residence Visa (Education Category)## Fees & Processing Time
* **Fee:** USD 200 (For a year or part thereof).
* **Processing Time / Validity:** Valid for one year (or time period recommended by the institution). It is renewable.
""","""
Residence Visa (Education Category)## Contact Information
* **Office:** Department of Immigration & Emigration, "Suhurupaya", Battaramulla.
""","""
Residence Visa (Education Category)## Common Issues & Solutions
* **Problem:** Visa Renewal (for non-university students).
* **Solution:** Requires a new request letter, recommendation from the Embassy, new bank encashment receipts (USD 1,500), and a performance report for the current year.
""","""
Residence Visa (Employment Category)## Eligibility Criteria
* Expatriate personnel employed in projects under the Board of Investment (BOI).
* Professional personnel for projects approved by the government.
* Personnel employed at banks.
* Personnel employed in a registered Private Sector Company.
* Dependents (spouse and children) of the above.
""","""
Residence Visa (Employment Category)## Required Documents
* **General (All Categories):**
    1.  Request Letter from the company/institute.
    2.  A copy of the applicant's passport Bio Data Page.
    3.  Health Protection Plan (HPP) certificate.
    4.  Passport(s) of the applicant (and dependents).
* **Category-Specific:**
    * **BOI:** BOI Certificate, BOI Agreement, Company Registration, Board of Directors details, Recommendation from BOI.
    * **Govt. Project:** Recommendation of the Line Ministry, Project Agreement.
    * **Banks:** Recommendation from the Central Bank of Sri Lanka.
    * **Private Sector:** Company Registration, Board of Directors details, Article of Association, Recommendation from the Line Ministry.
""","""
Residence Visa (Employment Category)## Step-by-Step Procedure
1.  Obtain the Residence Visa application form (from Head Office or download).
2.  Obtain the mandatory Health Protection Plan (HPP) via `www.health.gov.lk/IHU`. (Note: Some categories, like Investors and Diplomats, are exempt).
3.  Gather all general and category-specific documents.
4.  Submit the application to the Department of Immigration & Emigration, Colombo.
""","""
Residence Visa (Employment Category)## Fees & Processing Time
* **Fee:** USD 200 (For a year or part thereof).
* **Processing Time / Validity:** Valid for one year (or the period recommended by the authority). It is renewable annually.
""","""
Residence Visa (Employment Category)## Contact Information
* **Office:** Department of Immigration & Emigration, "Suhurupaya", Battaramulla.
""","""
Residence Visa (Employment Category)## Common Issues & Solutions
* **Problem:** Visa Renewal.
* **Solution:** Applications for renewal must be submitted with a new recommendation from the respective Line Ministry, BOI, or Central Bank.
* **Problem:** Health Requirement.
* **Solution:** Applications will not be accepted without proof of the Health Protection Plan (HPP), unless the applicant is in an exempt category.
""","""
Residence Visa (Other Categories & Fees)## Eligibility Criteria
* This covers various other residence visa classes.
""","""
Residence Visa (Other Categories & Fees)## Required Documents
* Varies by category.
""","""
Residence Visa (Other Categories & Fees)## Step-by-Step Procedure
* Varies by category.
""","""
Residence Visa (Other Categories & Fees)## Fees & Processing Time
* **Annual Fee (or part thereof) per person:**
    * **USD 500:** Digital Nomad.
    * **USD 200:**
        * Investors
        * Medical Visa (and assistants/dependents)
        * Religious Workers / Clergy
        * Non-government institutions
        * "My Dream Home" Visa (Extension only)
        * "Resident Guest" Scheme (Extension only)
        * Court Appearance Visa
        * Port City Visa
        * Spouse/dependents of the above.
    * **USD 150:**
        * Dependent children of a non-Sri Lankan spouse.
        * Children of Sri Lankan parents who hold foreign passports.
    * **USD 100:**
        * Ex-Sri Lankans (whose citizenship ceased) and their dependents.
        * Registered Indians (Indo-Lanka Agreement) and their dependents.
        * Non-Sri Lankan spouse of a *demised* Sri Lankan (with employment approval).
        * Non-Sri Lankan spouse (living >10 years, with employment approval).
        * Non-Sri Lankan spouse (with children <18, with employment approval).
    * **Free:**
        * Foreign spouse who is married to a Sri Lankan citizen.
""","""
Residence Visa (Other Categories & Fees)## Contact Information
* **Office:** Department of Immigration & Emigration, "Suhurupaya", Battaramulla.
""","""
Residence Visa (Other Categories & Fees)## Common Issues & Solutions
* Not specified.
""","""
Resident Guest Scheme Visa (Extension Only)## Eligibility Criteria
* This service is **only for renewal/extension** for existing visa holders.
* **Investor Category:** Must prove satisfactory performance of the project and maintain a minimum balance of USD 250,000 in the RGFCA.
* **Professional Category:** Must prove project progress and a monthly remittance of USD 2,000 (plus USD 1,000 per dependent).
""","""
Resident Guest Scheme Visa (Extension Only)## Required Documents
* **Both Categories:**
    1.  Completed application.
    2.  Passport copies and two photos.
    3.  Medical Report (from IOM Sri Lanka).
    4.  Police Clearance Report (from Sri Lanka).
* **Investor Category:**
    1.  Proof of project performance.
    2.  Updated RGFCA Bank Statement (min. USD 250,000).
    3.  Tax clearance documents.
* **Professional Category:**
    1.  Request letter.
    2.  Proof of professional experience and project progress.
    3.  Updated Bank Statement (showing monthly remittances).
""","""
Resident Guest Scheme Visa (Extension Only)## Step-by-Step Procedure
1.  Gather all required documents for the specific category.
2.  Submit the renewal application to the visa branch (4th floor) of the Department.
""","""
Resident Guest Scheme Visa (Extension Only)## Fees & Processing Time
* **Fee:** USD 200 per person for a one-year period.
* **Validity:** One year (renewable).
""","""
Resident Guest Scheme Visa (Extension Only)## Contact Information
* **Office:** Visa Branch (4th Floor), Department of Immigration & Emigration, "Suhurupaya", Battaramulla.
""","""
Resident Guest Scheme Visa (Extension Only)## Common Issues & Solutions
* **Problem:** Terminating the investment/project and leaving Sri Lanka.
* **Solution:** Must provide two months' written notice to the Department before terminating the project, and 14 working days' written notice before departing the country.
""","""
Tourist Visa (via ETA)## Eligibility Criteria
* Bona-fide tourists visiting for:
    * Sightseeing / Holidaying
    * Visiting friends and relatives
    * Medical treatment (including Ayurveda and Yoga)
    * Participating in sports events, competitions, or cultural performances.
* Must not engage in any business, trade, or employment (paid or unpaid).
* **Free Visa Pilot Program:** Nationals of China, India, Indonesia, Russia, Thailand, Malaysia, and Japan are eligible for a 30-day single-entry free visa (until further notice).
""","""
Tourist Visa (via ETA)## Required Documents
1.  Valid passport (data page image required for online application).
""","""
Tourist Visa (via ETA)## Step-by-Step Procedure
1.  Apply for the Electronic Travel Authorization (ETA) online at the official website.
2.  Alternatively, applications can be submitted via a third party, at a Sri Lanka Overseas Mission, at the Head Office, or on arrival at the port of entry.
""","""
Tourist Visa (via ETA)## Fees & Processing Time
* **Validity:** 30 days, double entry.
* **Fee Structure:**
| Method | SAARC Countries | All Other Countries |
| :--- | :--- | :--- |
| Online / Third Party / Mission / Head Office | USD 20 | USD 50 |
| On Arrival | USD 25 | USD 60 |
* **Children under 12 years:** Free.
""","""
Tourist Visa (via ETA)## Contact Information
* **Website:** `https://eta.gov.lk`
* **Office:** Department of Immigration & Emigration, "Suhurupaya", Battaramulla.
""","""
Tourist Visa (via ETA)## Common Issues & Solutions
* **Problem:** Engaging in activities not allowed (e.g., business meetings).
* **Solution:** These activities are strictly prohibited on a tourist visa. A Business Visa must be obtained for such purposes.
""","""
Tourist Visa Extension## Eligibility Criteria
* Tourists who have entered Sri Lanka on an ETA or on-arrival visa (valid for 30 days) and wish to stay longer.
""","""
Tourist Visa Extension## Required Documents
* Not specified (likely requires passport and application form).
""","""
Tourist Visa Extension## Step-by-Step Procedure
* Applications for extension should be made to the Department of Immigration and Emigration.
""","""
Tourist Visa Extension## Fees & Processing Time
* **Fees:** Fees are charged for extensions *beyond* the initial 30 days.

| Country Group | 30 - 60 Days | 61 - 180 Days | 181 - 270 Days |
| :--- | :--- | :--- | :--- |
| **Group 1** (Austria, UK, Australia, Canada, Japan, Germany, France, etc.) | USD 100 | USD 150 | USD 200 |
| **United States of America** | USD 100 | USD 200 | *(Not specified)* |
| **Group 2** (Singapore, Maldives, Seychelles, Czech Rep.) | Free | USD 150 | USD 200 |
| **All Other Countries** (not in groups above) | USD 60 | USD 150 | USD 200 |

* **Processing Time:** Not specified.
""","""
Tourist Visa Extension## Contact Information
* **Office:** Department of Immigration & Emigration, "Suhurupaya", Battaramulla.
""","""
Tourist Visa Extension## Common Issues & Solutions
* **Problem:** Extending the "Free Visa" (for 7 nationals).
* **Solution:** The initial 30-day visa is free, but any **extension is available subject to payment** according to the standard fee schedule above.
""","""
Visa Overstay Penalties## Eligibility Criteria (Applicability)
* Applies to foreign nationals who remain in Sri Lanka after their visa's period of validity has expired.
""","""
Visa Overstay Penalties## Required Documents
* Not applicable for the penalty itself, but a valid passport is required for departure.
""","""
Visa Overstay Penalties## Step-by-Step Procedure
* **If overstay is 7 days or less:** The person may depart from Sri Lanka by paying only the applicable visa fee at the port of departure. No penalty is charged.
* **If overstay is 8 to 14 days:** The person must pay a penalty of **USD 250** in addition to the applicable visa fee.
* **If overstay is 15 days or more:** The person must pay a penalty of **USD 500** in addition to the applicable visa fee.
""","""
Visa Overstay Penalties## Fees & Processing Time
* **Fees (Penalty Only):**
    * **0-7 days overstay:** USD 0
    * **8-14 days overstay:** USD 250
    * **15+ days overstay:** USD 500
* **Processing Time:** Paid upon departure or when rectifying visa status.
""","""
Visa Overstay Penalties## Contact Information
* Implied: Port of Departure or Department of Immigration & Emigration Head Office.
""","""
Visa Overstay Penalties## Common Issues & Solutions
* Not specified.
""","""
Yellow Fever Vaccination Certificate Requirement## Eligibility Criteria (Applicability)
* This is a mandatory entry requirement for passengers who are from, or have transited through, specific countries in the African and South American regions.
* **African Region:** Includes Angola, Benin, Nigeria, Ethiopia, Ghana, Kenya, Sudan, Uganda, etc.
* **South American Region:** Includes Argentina, Bolivia, Brazil, Colombia, Ecuador, Peru, Venezuela, etc.
""","""
Yellow Fever Vaccination Certificate Requirement## Required Documents
* A valid Yellow Fever Vaccination Certificate.
""","""
Yellow Fever Vaccination Certificate Requirement## Step-by-Step Procedure
1.  The passenger must produce the certificate to the immigration and emigration counter at the Airport upon arrival.
""","""
Yellow Fever Vaccination Certificate Requirement## Fees & Processing Time
* Not applicable.
""","""
Yellow Fever Vaccination Certificate Requirement## Contact Information
* Port of Entry (Airport).
""","""
Yellow Fever Vaccination Certificate Requirement## Common Issues & Solutions
* **Problem:** Failure to produce the certificate.
* **Solution:** The passenger **will not be allowed to enter** Sri Lanka.
"""
]

chunk_embeddings = []
summary_embeddings = []
title_embedding = []
title = []
texts = []
summaries = []

print(f"Processing {len(chunks)} chunks...")

for i, chunk in enumerate(chunks):
    try:
        cleanedText = clean_text(chunk)
        cleanedTopic = clean_text(chunkTopics[i])
        
        print(f"Processing chunk {i + 1}/{len(chunks)}...")
        
        chunk_vector = client.feature_extraction(
            text=cleanedText,
            model="google/embeddinggemma-300m"
        )
        summary_result = client.summarization(
            text=cleanedText,
            #model="facebook/bart-large-cnn"
            model="google/pegasus-cnn_dailymail"
        )
        
        summary_text = summary_result.summary_text
        
        summary_vector = client.feature_extraction(
            text=summary_text,
            model="google/embeddinggemma-300m"
        )
        
        topic_vector = client.feature_extraction(
            text=cleanedTopic,
            model="google/embeddinggemma-300m"
        )
        
        chunk_embeddings.append(chunk_vector)
        summary_embeddings.append(summary_vector)
        title_embedding.append(topic_vector)
        texts.append(cleanedText)
        summaries.append(summary_text)
        title.append(cleanedTopic)
               
    except Exception as e:
        print(f"✗ Error processing chunk {i + 1}: {e}")
        continue

print(f"\n✓ Completed! Successfully processed {len(texts)} chunks")


Processing 96 chunks...
Processing chunk 1/96...
Processing chunk 2/96...
Processing chunk 2/96...
Processing chunk 3/96...
Processing chunk 3/96...
Processing chunk 4/96...
Processing chunk 4/96...
Processing chunk 5/96...
Processing chunk 5/96...
Processing chunk 6/96...
Processing chunk 6/96...
Processing chunk 7/96...
Processing chunk 7/96...
Processing chunk 8/96...
Processing chunk 8/96...
Processing chunk 9/96...
Processing chunk 9/96...
Processing chunk 10/96...
Processing chunk 10/96...
Processing chunk 11/96...
Processing chunk 11/96...
Processing chunk 12/96...
Processing chunk 12/96...
Processing chunk 13/96...
Processing chunk 13/96...
Processing chunk 14/96...
Processing chunk 14/96...
Processing chunk 15/96...
Processing chunk 15/96...
Processing chunk 16/96...
Processing chunk 16/96...
Processing chunk 17/96...
Processing chunk 17/96...
Processing chunk 18/96...
Processing chunk 18/96...
Processing chunk 19/96...
Processing chunk 19/96...
Processing chunk 20/96...
Proce

In [41]:
for i, chunk in enumerate(chunks):
    database.upsert(
        collection_name="PassportKnowledgeBase",
        points=[
            models.PointStruct(
                id=i,
                vector={
                    "chunk_vector": chunk_embeddings[i],
                    "summary_vector": summary_embeddings[i],
                    "title_vector": title_embedding[i]
                },
                payload={
                    "text": texts[i],
                    "summary": summaries[i],
                    "title": title[i]
                }
            )
        ]
    )
